<a href="https://colab.research.google.com/github/GKD-stack/DSC180A-Methodology-1/blob/main/DSC180A_Week5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!! pip install pandas_plink

In [ ]:
from pandas_plink import read_plink
import pandas_plink as pdplink
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


# Part 1: Scaling Up eQTL Analysis

In [ ]:
#Load genotype data
(bim, fam, bed) = read_plink('/content/drive/My Drive/Colab Notebooks/1000G.EUR.1')

Mapping files: 100%|██████████| 3/3 [00:01<00:00,  1.58it/s]


In [ ]:
#load expression data
expr_data = pd.read_table("/content/drive/My Drive/Colab Notebooks/exp.txt", sep="\t")
expr_data = expr_data[expr_data['Chr'] == '1']
expr_data['TargetID'] = expr_data['TargetID'].str.split('.').str[0]
expr_data['Gene_Symbol'] = expr_data['Gene_Symbol'].str.split('.').str[0]
expr_data = expr_data.drop(columns=['TargetID','Chr', 'Coord'])
expr_data

In [ ]:
fam

In [ ]:
# Perform a cis-eQTL analysis for all genes on a chromosome of your choosing using 1000 Genome data.
# Didn't get to finish this, it was still running.
c1_pvals = {}

for i, (gene_symbol, gene_data) in enumerate(expr_data.groupby('Gene_Symbol')):
    gene_vals = gene_data.drop(columns=['Gene_Symbol'])

    # Filter and sort fam and expression data
    fam_ids = fam['fid'].tolist()
    expression_ids = gene_vals.columns.tolist()
    common_ids = set(fam_ids) & set(expression_ids)
    fam_df_filtered = fam[fam['fid'].isin(common_ids)]
    expression_df_filtered = gene_vals[common_ids]

    fam_df_sorted = fam_df_filtered.sort_values(by='fid')
    expression_df_sorted = expression_df_filtered[sorted(expression_df_filtered.columns.tolist())]

    # Filter bed
    expression_ids = expression_df_sorted.columns.tolist()
    fam_ids = fam_df_sorted.iloc[:, 1].tolist()
    indices = [fam_ids.index(exp_id) for exp_id in expression_ids if exp_id in fam_ids]
    bed_filtered = bed[:, indices]

    # Calculate p-values
    p_values = []
    expression_values = expression_df_sorted.iloc[0, :].values
    for snp_index in range(bed_filtered.shape[0]):
        genotype = bed_filtered[snp_index, :]

        if len(genotype) != len(expression_values):
            continue

        X = sm.add_constant(genotype)
        model = sm.OLS(expression_values, X)
        results = model.fit()
        p_values.append(results.pvalues[0])

    p_values = np.array(p_values)
    c1_pvals[gene_symbol] = p_values

In [ ]:
summary_stats.head()

,ID,A1,A2,BETA,P
0,rs3094315,A,G,-0.145844,0.792854
1,rs3131972,G,A,-0.184072,0.739817
2,rs3131969,G,A,-0.041249,0.943262
3,rs1048488,T,C,-0.149048,0.785585
4,rs3115850,C,T,-0.149048,0.785585


In [ ]:
df_transposed = expression_df_filtered.transpose().reset_index()
# Rename the columns to 'IID' and 'Phenotype'
df_transposed.columns = ['IID', 'Phenotype']
# Save the transposed and correctly formatted data to a text file
df_transposed.to_csv('/content/drive/My Drive/Colab Notebooks/exp_agrn.txt', sep='\t', index=False, header=True)
df_transposed

,IID,Phenotype
0,HG00376,0.531959
1,NA06986,0.859708
2,NA20585,0.494037
3,NA07048,0.437840
4,HG00102,0.708518
...,...,...
339,HG00103,0.860887
340,NA12890,1.511788
341,NA20778,0.955657
342,HG02215,0.507654


# Part 2: Building a PRS
For this section, I built a PRS for the AGRN gene. After the discussion, I finished up this part and found a gene that appeared to have more promising p values and did the cross validation on that for part 3.  

This was the plink command for clumping.
```
  plink ^
  --bfile new_filtered_dataset ^
  --pheno new_exp_agrn.txt ^
  --clump-p1 1 ^
  --clump-r2 0.1 ^
  --clump-kb 250 ^
  --clump basic.txt ^
  --clump-snp-field ID ^
  --clump-field P ^
  --out EUR
```

Then I got a list of valid SNPs (ran in cmd).


```
Get-Content EUR.clumped | Select-Object -Skip 1 | ForEach-Object { ($_ -split "\s+")[3] } > EUR.valid.snp

```
Then I got the p values for the corresponding SNPs (ran in powershell).
```
Get-Content basic.txt | ForEach-Object {
    $parts = $_ -split '\s+'
    "$($parts[0]) $($parts[1])"
} > SNP.pvalue
```
Then I parsed the summary stats file for ids, beta, and p values.
```
plink --bfile new_filtered_dataset --score basic.txt 1 2 4 header --out EUR
```
Then I put this code in cmd for the PSR.
```

plink --bfile new_filtered_dataset --score basic.txt 1 2 4 header --out EUR

```

plink --bfile new_filtered_dataset --score basic.txt 1 2 4 header --q-score-range range_list.txt SNP.pvalue --out EUR

did not work





# Part 3: Cross Validation

## Part 3.1: Sampling
Randomly sample 80% of individuals and perform the cis-eQTL analysis for one gene (you can use the –keep flag for plink to select certain sample IDs)

1. Make a coordinates file


```
chromosome start_position end_position gene_ID score strand
```

2. Filter existing data


```
plink2 –bfile LDREF/1000G.EUR.1 –extract bed1 test_coordinates.txt –out 1000G.EUR.1.Test –make-bed
```



Run an eQTL analysis for a single gene of your chosing. For this gene, select only the SNPs within +/- 500 kb of the gene body (you can use gene_annot.txt.gz as the master list of all genes genome-wide). You can select these SNPs by using plink with the –extract bed1 flag. For example: plink2 –bfile 1000G.EUR.15 –extract bed1 $my_coord –out 1000G.EUR.15.GeneX –make-bed (in this case, $my_coord is a single line 6-column tab delimited text file with the genomic coordinates of the +/- 500 kb window around your gene of interest, for example, chr15 200000 1200000 mygene 0 +

In [ ]:
#get genotype data
(bim, fam, bed) = read_plink('/content/drive/My Drive/Colab Notebooks/1000G.EUR.1.Test')

Mapping files: 100%|██████████| 3/3 [00:01<00:00,  1.99it/s]


In [ ]:
#clean it just to get 1 gene
expr_data = pd.read_table("/content/drive/My Drive/Colab Notebooks/exp.txt", sep="\t")
expr_data['TargetID'] = expr_data['TargetID'].str.split('.').str[0]
expr_data = expr_data[expr_data["TargetID"] == "ENSG00000134222"] #changed it
expr_data = expr_data.drop(columns=['TargetID','Gene_Symbol', 'Chr', 'Coord'])

#filter fam
fam_ids = fam['fid'].tolist()
expression_ids = expr_data.columns.tolist()
common_ids = set(fam_ids) & set(expression_ids)
fam_df_filtered = fam[fam['fid'].isin(common_ids)]

#reshape exp
expr_data = expr_data[common_ids]
expr_data = expr_data.transpose().reset_index()
expr_data.columns = ['IID', 'Phenotype']

#filter fam
fam_df_filtered = fam_df_filtered.drop(columns='i')
fam_df_filtered = fam_df_filtered.sort_values(by='fid')

expr_data['FID'] = expr_data['IID']
expr_data = expr_data.sort_values(by='IID')
expr_data= expr_data[['IID', 'FID', 'Phenotype']]#make sure IDs line up

if fam_df_filtered['fid'].tolist() == expr_data['IID'].tolist():
    print("Yes")
else:
    print("No")

#filter bed
expression_ids = expr_data['IID'].tolist()
fam_ids = fam_df_filtered.iloc[:, 1].tolist()
indices = [fam_ids.index(exp_id) for exp_id in expression_ids if exp_id in fam_ids]
bed_filtered = bed[:, indices]

combined_data = expr_data.merge(fam_df_filtered, left_on='IID', right_on='fid')

from sklearn.model_selection import train_test_split
# Split the data into training (80%), validation (10%), and test (10%)
train, temp = train_test_split(combined_data, test_size=0.2, random_state=42)
validation, test = train_test_split(temp, test_size=0.5, random_state=42)

#split a df back
exp_cols = ['IID', 'FID', 'Phenotype']
fam_cols = ['fid', 'iid','father','mother','gender','trait']
train_exp = train[exp_cols]
train_fam = train[fam_cols]
validation_exp = validation[exp_cols]
validation_fam = validation[fam_cols]
test_exp = test[exp_cols]
test_fam = test[fam_cols]
# validation_exp.to_csv('/content/drive/My Drive/Colab Notebooks/test2.fam', sep='\t', index=False)
# validation_fam.to_csv('/content/drive/My Drive/Colab Notebooks/test2.bim', sep=' ', index=False)

Yes


<ipython-input-65-a40031c6e8f7>:14: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  expr_data = expr_data[common_ids]


In [ ]:
train_exp

In [ ]:
x = train_fam[['fid','iid']]
x.columns = ['FID','IID']
x.to_csv('/content/drive/My Drive/Colab Notebooks/training_sample_ids.txt', sep='\t', index=False, header=False)
train_exp.to_csv('/content/drive/My Drive/Colab Notebooks/phenotype_file.txt', sep='\t', index=False, header=False)


eQTL analysis


```
plink --bfile 1000G.EUR.1.Test --keep training_sample_ids.txt --linear --pheno phenotype_file.txt --out test_stats

```



In [ ]:
train_indices = train_fam[train_fam['iid'].isin(train_exp['IID'])].index.tolist()
bed_filtered_train = bed_filtered[:, train_indices]

p_values = []
expression_values = expr_data['Phenotype'].values
betas = []
p_values = []

# Run eQTL analysis for each SNP
for snp_index in range(bed_filtered_train.shape[0]):
    genotype = bed_filtered[snp_index, :]
    X = sm.add_constant(genotype)
    model = sm.OLS(expression_values, X)
    results = model.fit()
    try:
        betas.append(results.params[1])  # Try to append the beta value
        p_values.append(results.pvalues[1])  # Try to append the p-value
    except IndexError:  # If IndexError is raised, append None
        betas.append(None)
        p_values.append(None)
# Add SNP information to the results
snp_info = bim[['snp', 'a0', 'a1']].values
summary_stats = pd.DataFrame({
    'ID': snp_info[:, 0],
    'A1': snp_info[:, 2],
    'A2': snp_info[:, 1],
    'BETA': betas,
    'P': p_values
})

summary_stats.columns = summary_stats.columns.str.strip()
summary_stats.to_csv("/content/drive/My Drive/Colab Notebooks/test_stats.txt", sep='\t', index=False)
summary_stats

,ID,A1,A2,BETA,P
0,rs2336016,G,A,-0.045479,0.717669
1,rs2336015,C,T,-0.030343,0.801833
2,rs10881518,C,T,-0.018633,0.878331
3,rs4970822,C,A,-0.107642,0.452618
4,rs4970821,T,C,-0.039160,0.758294
...,...,...,...,...,...
333,rs4268379,T,C,0.143755,0.203865
334,rs658092,C,T,-0.062368,0.657531
335,rs611945,A,G,0.211467,0.080783
336,rs611905,A,G,0.211467,0.080783


## Part 3.2: Run PRS Pipeline with Validation Set
Randomly sample half of the remaining 20% of individuals. With these people, run the PRS pipeline with several p-value thresholds: 1, 0.05, 0.01, 0.001, and 0.00001. Correlate the predicted value of gene expression with the true value. Pick the p-value threshold that maximizes the prediction R2.

In [ ]:
#make summary stats for validation
validation_indices = validation_fam[validation_fam['iid'].isin(validation_exp['IID'])].index.tolist()
bed_filtered_validation = bed_filtered[:, validation_indices]
betas = []
p_values = []
expression_values = validation_exp['Phenotype'].values

# Run eQTL analysis for each SNP
for snp_index in range(bed_filtered_validation.shape[0]):
    genotype = bed_filtered_validation[snp_index, :]
    X = sm.add_constant(genotype)
    model = sm.OLS(expression_values, X)
    results = model.fit()
    try:
        betas.append(results.params[1])  # Try to append the beta value
        p_values.append(results.pvalues[1])  # Try to append the p-value
    except IndexError:  # If IndexError is raised, append None
        betas.append(None)
        p_values.append(None)
# Add SNP information to the results
snp_info = bim[['snp', 'a0', 'a1']].values
summary_stats = pd.DataFrame({
    'ID': snp_info[:, 0],
    'A1': snp_info[:, 2],
    'A2': snp_info[:, 1],
    'BETA': betas,
    'P': p_values
})
summary_stats

,ID,A1,A2,BETA,P
0,rs2336016,G,A,0.088557,0.850055
1,rs2336015,C,T,0.135421,0.747765
2,rs10881518,C,T,0.228955,0.585541
3,rs4970822,C,A,0.059601,0.901428
4,rs4970821,T,C,0.088557,0.850055
...,...,...,...,...,...
333,rs4268379,T,C,0.159161,0.701837
334,rs658092,C,T,0.233882,0.577996
335,rs611945,A,G,-0.059782,0.877385
336,rs611905,A,G,-0.059782,0.877385


In [ ]:
p_thresholds = [0.00001, 0.001, 0.01, 0.05, 1]
for threshold in p_thresholds:
    filtered_df = summary_stats[summary_stats['P'] <= threshold]
    filtered_df.to_csv(f'/content/drive/My Drive/Colab Notebooks/basic_threshold_{threshold}.txt', sep="\t", index=False)
    print(filtered_df)

I tried using --q-score-range to do all the ranges at once but it didnt work so i just ran these. Also it looks like this gene wasn't too significant either.

```
plink --bfile 1000G.EUR.1.Test --score "basic_threshold_0.00001.txt" 1 2 4 header --out "EUR_threshold_0.00001"
```
No alleles here

```
plink --bfile 1000G.EUR.1.Test --score "basic_threshold_0.001.txt" 1 2 4  
header --out "EUR_threshold_0.001"
```

No alleles here



```
plink --bfile 1000G.EUR.1.Test --score "basic_threshold_0.01.txt" 1 2 4 header --out "EUR_threshold_0.01"
```
no alleles here



```
plink --bfile 1000G.EUR.1.Test --score "basic_threshold_0.05.txt" 1 2 4 header --out "EUR_threshold_0.05"
```
no results written to file

```
plink --bfile 1000G.EUR.1.Test --score "basic_threshold_1.txt" 1 2 4 header --out "EUR_threshold_1"

```
332 loaded



In [ ]:
validation_exp

In [ ]:
profile_df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/EUR_threshold_1.profile', delim_whitespace=True)
profile_df

In [ ]:
from scipy.stats import linregress
merged_df = pd.merge(validation_exp, profile_df, on='IID')
slope, intercept, r_value, p_value, std_err = linregress(merged_df['Phenotype'], merged_df['SCORE'])
print(f'r squared: {r_value**2}')

r squared: 0.292194984101133


## Part 3.3: Run on Test Sample

Assess the PRS accuracy in the remaining individuals (none of these people have been used in the model fitting part). What is the final R2?

In [ ]:
test_indices = test_fam[test_fam['iid'].isin(test_exp['IID'])].index.tolist()
bed_filtered_test = bed_filtered[:, test_indices]
betas = []
p_values = []
expression_values = test_exp['Phenotype'].values

# Run eQTL analysis for each SNP
for snp_index in range(bed_filtered_test.shape[0]):
    genotype = bed_filtered_test[snp_index, :]
    X = sm.add_constant(genotype)
    model = sm.OLS(expression_values, X)
    results = model.fit()
    try:
        betas.append(results.params[1])  # Try to append the beta value
        p_values.append(results.pvalues[1])  # Try to append the p-value
    except IndexError:  # If IndexError is raised, append None
        betas.append(None)
        p_values.append(None)
# Add SNP information to the results
snp_info = bim[['snp', 'a0', 'a1']].values
summary_stats = pd.DataFrame({
    'ID': snp_info[:, 0],
    'A1': snp_info[:, 2],
    'A2': snp_info[:, 1],
    'BETA': betas,
    'P': p_values
})
summary_stats.to_csv(f'/content/drive/My Drive/Colab Notebooks/test_final.txt', sep="\t", index=False)



```
plink --bfile 1000G.EUR.1.Test --score "test_final.txt" 1 2 4 header --out "EUR_threshold_1_test"
```



In [ ]:
profile_df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/EUR_threshold_1_test.profile', delim_whitespace=True)
merged_df = pd.merge(test_exp, profile_df, on='IID')
slope, intercept, r_value, p_value, std_err = linregress(merged_df['Phenotype'], merged_df['SCORE'])
print(f'r squared: {r_value**2}')

r squared: 0.000167758618877606
